### Importing relevant packages

In [1]:
import tensorflow as tf
import numpy as np

### Loading drive on colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
train_path ='drive/MyDrive/train_data.npz'
validation_path ='drive/MyDrive/validation_data.npz'

### Loading our data

In [8]:
npz = np.load(train_path)
train_inputs,train_targets = npz['inputs'].astype('float'), npz['targets'].astype('int')
 

npz = np.load(validation_path)
validation_inputs, validation_targets = npz['inputs'].astype('float'), npz['targets'].astype('int')

### Defining vocab size

In [5]:
VOCAB_SIZE= 482112 +1

### Outlining our model

In [68]:
model = tf.keras.models.Sequential([    
    tf.keras.layers.Embedding(input_dim=VOCAB_SIZE,output_dim=16,input_length=20),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(18, activation='softmax')
])

In [69]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 20, 16)            7713808   
                                                                 
 flatten_11 (Flatten)        (None, 320)               0         
                                                                 
 dense_24 (Dense)            (None, 32)                10272     
                                                                 
 dense_25 (Dense)            (None, 18)                594       
                                                                 
Total params: 7,724,674
Trainable params: 7,724,674
Non-trainable params: 0
_________________________________________________________________


In [70]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [71]:
early_stopping = tf.keras.callbacks.EarlyStopping(
monitor='val_accuracy',
patience=2,
restore_best_weights=True)

### Fitting our model

In [72]:
history = model.fit(train_inputs,
                    train_targets,
                    epochs=20,
                    validation_data=(validation_inputs, validation_targets), 
                    validation_steps=30,
                    shuffle=True,
                    batch_size=1024,
                    callbacks=[early_stopping])

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


403/403 [==============================] - 3s 8ms/step - loss: 2.4121 - accuracy: 0.2341 - val_loss: 1.9713 - val_accuracy: 0.3672
Epoch 2/20
403/403 [==============================] - 3s 7ms/step - loss: 1.6689 - accuracy: 0.4711 - val_loss: 1.7156 - val_accuracy: 0.4600
Epoch 3/20
403/403 [==============================] - 3s 7ms/step - loss: 1.2175 - accuracy: 0.6289 - val_loss: 1.6989 - val_accuracy: 0.4834
Epoch 4/20
403/403 [==============================] - 3s 7ms/step - loss: 0.8742 - accuracy: 0.7464 - val_loss: 1.8084 - val_accuracy: 0.4823
Epoch 5/20
403/403 [==============================] - 3s 7ms/step - loss: 0.6317 - accuracy: 0.8240 - val_loss: 1.9798 - val_accuracy: 0.4705


### Evaluating model

In [59]:
model.evaluate(validation_inputs,validation_targets)

1432/1432 [==============================] - 3s 2ms/step - loss: 1.7189 - accuracy: 0.4819


[1.7189421653747559, 0.48192930221557617]

### Saving our model

In [60]:
model.save('model.h5') 